In [43]:
# Parameters 
project = 'PowerBIDemo'
project_path = f'../src/{project}'

In [ ]:
# Libraries
import pyfabricops as pf
pf.enable_notebook_logging()
pf.set_auth_provider('env') 

import json
import subprocess

In [61]:
# Relative paths
branch = pf.get_current_branch() 
branches_path = '../branches.json'
workspace_suffix = pf.get_workspace_suffix(path=branches_path)  
workspace_name = project + workspace_suffix
print(f'Branch: {branch}, Workspace: {workspace_name}') 

Branch: main, Workspace: PowerBIDemo-PRD


In [47]:
# Retrienving the workspace_id from config.json in the project_path for better performance
config_path = f'{project_path}/config.json'
with open(config_path, 'r') as f:
  config_content = json.load(f)
  config = config_content[branch]  
  workspace_id = config[project]['workspace_config']['workspace_id'] 
print(f'Workspace ID: {workspace_id}')

Workspace ID: f4bb753f-4b0a-4f38-b0c1-e13cf18e3b1b


In [48]:
# Deploy the folders first.
pf.deploy_folders(
	workspace=workspace_id, 
	project_path=project_path,
	branches_path=branches_path,
)  

pyfabricops._folders INFO: Found 2 folders containing Fabric artifacts


pyfabricops._folders INFO: Created folders for workspace f4bb753f-4b0a-4f38-b0c1-e13cf18e3b1b.
pyfabricops._folders INFO: Updated folders config for workspace "PowerBIDemo" in branch "main"
pyfabricops._folders INFO: Folders configuration successfully written to ../src/PowerBIDemo\config.json.


In [ ]:
# Deploy the calendar dataflow gen1
# Due a conflict with others dataflow generations and limitations with folders we need deploy it separately.
pf.deploy_dataflow_gen1(
	workspace=workspace_id, 
	path=f'{project_path}/workspace/Calendar.Dataflow',
)  

In [ ]:
# Export dataflow config
# Dataflows gen1 don't have folders support on export.
# They arrive in the root of the workspace.
pf.export_all_dataflows_gen1(
	workspace=workspace_id,
    project_path=project_path,
	branches_path=branches_path,
)

In [51]:
# Read the config.json retriving the parameters for the semantic model
with open(config_path, 'r') as f:
  config_content = json.load(f)
  config = config_content[branch]
  semantic_models_config = config[project]['semantic_models']

In [55]:
# Deploying the semantic models
import glob 
import re

for semantic_model_path in glob.glob(f'{project_path}/**/*.SemanticModel', recursive=True):
	print(f'Processing semantic model: {semantic_model_path}')
	semantic_model_name = semantic_model_path.replace('\\', '/').split('/')[-1].split('.SemanticModel')[0] 
	print(f'Deploying semantic model name: {semantic_model_name}')  

	# Retrieving the semantic model config
	semantic_model_config = semantic_models_config[semantic_model_name]
	semantic_model_parameters = semantic_model_config['parameters']
	# Define your new values here
	new_server = semantic_model_parameters['Server']['Value']
	new_database = semantic_model_parameters['Database']['Value']
	new_calendar_workspace_id = semantic_model_parameters['CalendarWorkspaceId']['Value']
	new_calendar_dataflow_id = semantic_model_parameters['CalendarDataflowId']['Value']

	# Read the expressions file
	expressions_path = f'{semantic_model_path}//definition/expressions.tmdl'
	with open(expressions_path, 'r') as f:
		expressions = f.read()

	# Dictionary with variable mappings
	variables_to_replace = {
		'Server': new_server,
		'Database': new_database,
		'CalendarWorkspaceId': new_calendar_workspace_id,
		'CalendarDataflowId': new_calendar_dataflow_id
	}
	
	# Start with original expressions
	expressions_updated = expressions

	# Replace each variable using regex
	for variable_name, new_value in variables_to_replace.items():
		# Pattern to match: VariableName = "any_value"
		pattern = rf'{variable_name}\s*=\s*"[^"]*"'
		replacement = f'{variable_name} = "{new_value}"'
		expressions_updated = re.sub(pattern, replacement, expressions_updated)
		print(f"Replaced {variable_name} with: {new_value}")

	# Write the updated expressions back to the file
	with open(expressions_path, 'w', encoding='utf-8') as f:
		f.write(expressions_updated)


Processing semantic model: ../src/PowerBIDemo\workspace\Main\Sales.SemanticModel
Deploying semantic model name: Sales
Replaced Server with: PEZZOTT
Replaced Database with: AdventureWorksDW2022-PRD
Replaced CalendarWorkspaceId with: f4bb753f-4b0a-4f38-b0c1-e13cf18e3b1b
Replaced CalendarDataflowId with: e519fe9b-2f21-418e-b297-3e6af3de0c2d

=== EXPRESSIONS UPDATED ===
expression Server = "PEZZOTT" meta [IsParameterQuery=true, Type="Text", IsParameterQueryRequired=false]
	lineageTag: fbf210cc-61ab-47ab-bee6-b95993d2b482

	annotation PBI_ResultType = Text

expression Database = "AdventureWorksDW2022-PRD" meta [IsParameterQuery=true, Type="Text", IsParameterQueryRequired=false]
	lineageTag: 30ca4587-6ee7-4c47-bb17-22f07992ac68

	annotation PBI_ResultType = Text

	annotation PBI_NavigationStepName = Navigation

expression CalendarWorkspaceId = "f4bb753f-4b0a-4f38-b0c1-e13cf18e3b1b" meta [IsParameterQuery=true, Type="Text", IsParameterQueryRequired=false]
	lineageTag: 540e979e-d8f5-4bc9-bb51-

In [ ]:
# Deploy all the semantic models
pf.deploy_all_semantic_models(
    workspace=workspace_id, 
    project_path=project_path,
    branches_path=branches_path,
)

In [57]:
# Retrieving the semantic_model_details
pf.export_all_semantic_models(
    workspace=workspace_id, 
    project_path=project_path,
    branches_path=branches_path,
)

pyfabricops._semantic_models INFO: Found existing config file at ../src/PowerBIDemo\config.json, merging workspace config...
pyfabricops._utils INFO: Item definition unpacked to ../src/PowerBIDemo\workspace\Main/Sales.SemanticModel


In [58]:
# For each report, we will define the report attached to the semantic model and deploy it.
import glob 
import re

for report_path in glob.glob(f'{project_path}/**/*.Report', recursive=True):
	print(f'Processing report: {report_path}')
	report_name = report_path.replace('\\', '/').split('/')[-1].split('.Report')[0] 
	print(f'Deploying report: {report_name}')

	with open(f'{report_path}/definition.pbir', 'r') as f:
		report_definition = json.load(f)

	dataset_reference = report_definition['datasetReference']

	if 'byPath' in dataset_reference:
		dataset_path = dataset_reference['byPath']['path'] 
		dataset_name = dataset_path.split('/')[-1].split('.SemanticModel')[0]

	elif 'byConnection' in dataset_reference:
		text_to_search = dataset_reference['byConnection']['connectionString']
		# Capture the value after "initial catalog="
		match = re.search(r'initial catalog=([^;]+)', text_to_search)
		if match:
			dataset_name = match.group(1)

	print(f'Semantic model: {dataset_name}')
	
	# Search for the semantic model in the config.json
	with open(config_path, 'r') as f:
		config_content = json.load(f)
	config = config_content[branch]
	semantic_model_id = config[project]['semantic_models'][dataset_name]['id'] 
	print(f'Semantic Model ID: {semantic_model_id}')  
	
	# Replace the definition.pbir with the updated template
	with open(f'../template_report_definition.pbir', 'r', encoding='utf-8') as f:
		report_definition_template = f.read()

	report_definition_updated = report_definition_template.replace('{workspace_name}', workspace_name)
	report_definition_updated = report_definition_updated.replace('{semantic_model_name}', dataset_name)
	report_definition_updated = report_definition_updated.replace('{semantic_model_id}', semantic_model_id)
	# print(report_definition_updated)    

	with open(f'{report_path}/definition.pbir', 'w', encoding='utf-8') as f:
		f.write(report_definition_updated)
	
	# Deploy the report
	pf.deploy_report(
		workspace=workspace_id, 
		display_name=report_name,
		project_path=project_path,
		branches_path=branches_path,
	)
	
	# Write back the original report_definition for the definition.pbir
	with open(f'{report_path}/definition.pbir', 'w', encoding='utf-8') as f:
		json.dump(report_definition, f, indent=2)


Processing report: ../src/PowerBIDemo\workspace\Customers\Customers.Report
Deploying report: Customers
Semantic model: Sales
Semantic Model ID: 5d72e3ae-64c5-4061-bdce-203643dc6362


pyfabricops._reports INFO: Creating new report: Customers
pyfabricops._reports INFO: Successfully created report 'Customers'


Processing report: ../src/PowerBIDemo\workspace\Main\Sales.Report
Deploying report: Sales
Semantic model: Sales
Semantic Model ID: 5d72e3ae-64c5-4061-bdce-203643dc6362


pyfabricops._reports INFO: Creating new report: Sales
pyfabricops._reports INFO: Successfully created report 'Sales'


In [59]:
# Extract the reports from the workspace to update config.json if you want to update the config.json with the latest reports.
# If you constantly update the reports, you can skip this step because the definition.pbir is binded locally to semantic model.
pf.export_all_reports(
	workspace=workspace_id, 
	project_path=project_path,
	branches_path=branches_path,
)

pyfabricops._reports INFO: Found existing config file at ../src/PowerBIDemo\config.json, merging workspace config...
pyfabricops._utils INFO: Item definition unpacked to ../src/PowerBIDemo\workspace\Customers/Customers.Report
pyfabricops._reports INFO: Found existing config file at ../src/PowerBIDemo\config.json, merging workspace config...
pyfabricops._utils INFO: Item definition unpacked to ../src/PowerBIDemo\workspace\Main/Sales.Report
pyfabricops._dataflows_gen1 INFO: Exported dataflow Calendar to ../src/PowerBIDemo\workspace\Calendar.Dataflow.
pyfabricops._dataflows_gen1 INFO: Found existing config file at ../src/PowerBIDemo\config.json, merging workspace config...


In [ ]:
# Finally, we will commit the changes to git.

message = f'{project}: Changes commited.'

# Git commit and checkout to new branch to start development
# Add  the project path to git
subprocess.run(['git', 'add', project_path])

# Commit the changes
subprocess.run(['git', 'commit', '-m', message]) 

CompletedProcess(args=['git', 'commit', '-m', 'feat: PowerBIDemo: Development done for the semantic model and reports'], returncode=0)